In [1]:
!pip install -r https://raw.githubusercontent.com/datamllab/automl-in-action-notebooks/master/requirements.txt

In [2]:
import keras_tuner as kt


class MyTuner(kt.RandomSearch):
    def run_trial(self, trial, *args, **kwargs):
        hp = trial.hyperparameters
        x = hp.Float("x", -1.0, 1.0)
        return x * x + 1


tuner = MyTuner(max_trials=20)
tuner.search()
tuner.results_summary()

Trial 20 Complete [00h 00m 00s]
default_objective: 1.0198333738835208

Best default_objective So Far: 1.0026852807069282
Total elapsed time: 00h 00m 00s
Results summary
Results in ./untitled_project
Showing 10 best trials
Trial summary
Hyperparameters:
x: -0.0518196941994864
Score: 1.0026852807069282
Trial summary
Hyperparameters:
x: -0.0887713850034304
Score: 1.0078803587954273
Trial summary
Hyperparameters:
x: 0.12735132107069558
Score: 1.0162183589784515
Trial summary
Hyperparameters:
x: -0.13106783806824995
Score: 1.017178778175885
Trial summary
Hyperparameters:
x: -0.14083101179612667
Score: 1.0198333738835208
Trial summary
Hyperparameters:
x: 0.2582831674929471
Score: 1.0667101946101898
Trial summary
Hyperparameters:
x: -0.30974289344068207
Score: 1.0959406600370056
Trial summary
Hyperparameters:
x: -0.4117586407212144
Score: 1.1695451782085822
Trial summary
Hyperparameters:
x: 0.4904150980174993
Score: 1.2405069683635135
Trial summary
Hyperparameters:
x: -0.5009246474246642
Scor

In [3]:
import os
import pickle
import tensorflow as tf
import kerastuner as kt


class ShallowTuner(kt.Tuner):
    def __init__(self, oracle, hypermodel, **kwargs):
        super(ShallowTuner, self).__init__(
            oracle=oracle, hypermodel=hypermodel, **kwargs
        )

    def search(self, X, y, validation_data):
        """performs hyperparameter search."""
        return super(ShallowTuner, self).search(X, y, validation_data)

    def run_trial(self, trial, X, y, validation_data):
        model = self.hypermodel.build(trial.hyperparameters)  # build the model
        model.fit(X, y)  # fit the model
        X_val, y_val = validation_data  # get the validation data
        eval_score = model.score(X_val, y_val)  # evaluate the model
        self.save_model(trial.trial_id, model)  # save the model to disk
        return {"score": eval_score}

    def save_model(self, trial_id, model, step=0):
        """save the model with pickle"""
        fname = os.path.join(self.get_trial_dir(trial_id), "model.pickle")
        with tf.io.gfile.GFile(fname, "wb") as f:
            pickle.dump(model, f)

    def load_model(self, trial):
        """load the model with pickle"""
        fname = os.path.join(self.get_trial_dir(trial.trial_id), "model.pickle")
        with tf.io.gfile.GFile(fname, "rb") as f:
            return pickle.load(f)



<ipython-input-3-f8566eac545a>:4: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


In [4]:
from sklearn.datasets import load_digits

# Load the hand-written digits dataset
digits = load_digits()

# Get the images and corresponding labels
images, labels = digits.images, digits.target
images.shape, labels.shape

# reshape images to vectors
n_samples = len(images)
X = images.reshape((n_samples, -1))

# Split data into train and test subsets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, labels, test_size=0.2, shuffle=False
)
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.2, shuffle=False
)


In [5]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from kerastuner.engine import hyperparameters as hp


def build_model(hp):
    model_type = hp.Choice("model_type", ["svm", "random_forest"])
    if model_type == "svm":
        with hp.conditional_scope("model_type", "svm"):
            model = SVC(
                C=hp.Float("C", 1e-3, 10, sampling="linear", default=1),
                kernel=hp.Choice("kernel_type", ["linear", "rbf"], default="linear"),
                random_state=42,
            )
    elif model_type == "random_forest":
        with hp.conditional_scope("model_type", "random_forest"):
            model = RandomForestClassifier(
                n_estimators=hp.Int("n_estimators", 10, 200, step=10),
                max_depth=hp.Int("max_depth", 3, 10),
            )
    else:
        raise ValueError("Unrecognized model_type")
    return model


my_sklearn_tuner = ShallowTuner(
    oracle=kt.oracles.RandomSearch(
        objective=kt.Objective("score", "max"), max_trials=10, seed=42
    ),
    hypermodel=build_model,
    overwrite=True,
    project_name="my_sklearn_tuner",
)

my_sklearn_tuner.search(X_train, y_train, validation_data=(X_val, y_val))


Trial 10 Complete [00h 00m 00s]
score: 0.9826388888888888

Best score So Far: 0.9826388888888888
Total elapsed time: 00h 00m 03s


In [6]:
# Evaluate the best discovered model
from sklearn.metrics import accuracy_score

best_model = my_sklearn_tuner.get_best_models(1)[0]
best_model.fit(X_train, y_train)
y_pred_test = best_model.predict(X_test)
test_acc = accuracy_score(y_test, y_pred_test)
print("The prediction accuracy on test set: {:.2f} %".format(test_acc * 100))


The prediction accuracy on test set: 95.00 %
